# A two asset Monte Carlo

Pre-requisites: Notebook 1.1, 1.2, 2.1

In this notebook we will modify our custom MC Model to handle two assets, with different volatilities, and a flat correlation.

Let us start with the imports.

In [ ]:
import numpy as np
from math import sqrt
from qablet.base.mc import MCModel, MCStateBase  # The two base classes we need for our custom model
from numpy.random import Generator, SFC64
from qablet.base.utils import Forwards
from qablet_contracts.equity.vanilla import option_timetable
from qablet_contracts.timetable import timetable_from_dicts

State class for a two asset Black-Scholes model. In the dataset it expects a component `BS2` to contain the model parameters:
 - `ASSET1` and `ASSET2`: the name of the assets
 - `VOL1` and `VOL2`: the volatilities
 - `CORR`: the correlation between the two assets

In [ ]:
# Monte Carlo Pricer for BS (flat vol) Model with two assets

# Define a class for the state of a two asset BS MC process
class BS2State(MCStateBase):
    def __init__(self, timetable, dataset):

        super().__init__(timetable, dataset)

        # fetch the model parameters from the dataset
        modeldata = dataset["BS2"]
        self.asset1 = modeldata["ASSET1"]
        self.asset_fwd1 = Forwards(dataset["ASSETS"][self.asset1])
        self.spot1 = self.asset_fwd1.forward(0)
        self.vol1 = modeldata["VOL1"]

        self.asset2 = modeldata["ASSET2"]
        self.asset_fwd2 = Forwards(dataset["ASSETS"][self.asset2])
        self.spot2 = self.asset_fwd2.forward(0)
        self.vol2 = modeldata["VOL2"]

        self.corr = modeldata["CORR"]
        self.corr_sup = sqrt(1 - self.corr * self.corr)

        # Initialize the arrays
        self.n = dataset["MC"]["PATHS"]
        self.rng = Generator(SFC64(dataset["MC"]["SEED"]))
        self.x1_vec = np.zeros(self.n)  # process x1 (log stock)
        self.x2_vec = np.zeros(self.n)  # process x2 (log stock)

        self.cur_time = 0

    def advance(self, new_time):
        """Update x_vec in place when we move simulation by time dt."""

        dt = new_time - self.cur_time
        if dt < 1e-10:
            return


        drift1 = self.asset_fwd1.rate(new_time, self.cur_time)
        drift2 = self.asset_fwd2.rate(new_time, self.cur_time)

        # generate the random numbers and advance the log stock process
        dz1_vec = self.rng.standard_normal(self.n)
        dz2_vec = self.rng.standard_normal(self.n) * self.corr_sup + self.corr * dz1_vec

        self.x1_vec += (drift1 - self.vol1 * self.vol1 / 2.0) * dt
        self.x1_vec += dz1_vec * sqrt(dt) * self.vol1

        self.x2_vec += (drift2 - self.vol2 * self.vol2 / 2.0) * dt
        self.x2_vec += dz2_vec * sqrt(dt) * self.vol2

        self.cur_time = new_time

    def get_value(self, unit):
        """Return the value of the unit at the current time.
        This model uses black scholes model for one asset, return its value using the simulated array.
        For any other asset that may exist in the timetable, just return the default implementation in
        the model base (i.e. simply return the forwards)."""

        if unit == self.asset1:
            return self.spot1 * np.exp(self.x1_vec)
        elif unit == self.asset2:
            return self.spot2 * np.exp(self.x2_vec)
        else:
            return None


In [ ]:
class BS2Model(MCModel):

    def state_class(self):
        return BS2State

Sample market parameters

In [ ]:
times = np.array([0.0, 5.0])
rates = np.array([0.04, 0.04])
discount_data = ("ZERO_RATES", np.column_stack((times, rates)))

spot = 2900
div_rate = 0.01
fwds = 2900 * np.exp((rates - div_rate) * times)
spx_data = ("FORWARDS", np.column_stack((times, fwds)))
rut_data = ("FORWARDS", np.column_stack((times, 0.5 * fwds)))  # let's say rut is half of spx

dataset = {
    "MC": {
        "PATHS": 100_000,
        "TIMESTEP": 1 / 250,
        "SEED": 1,
        "RETURN_PV_VEC": True
    },
    "BASE": "USD",
    "ASSETS": {"USD": discount_data, "SPX": spx_data, "RUT": rut_data},
    "BS2": {
        "ASSET1": "SPX",
        "VOL1": 0.175,
        "ASSET2": "RUT",
        "VOL2": 0.175,
        "CORR": 1.0
    }
}


In [ ]:
strike = 2800
timetable = option_timetable("USD", "SPX", strike=strike, maturity=1.0, is_call=True)
print(timetable["events"].to_pandas())

  track  time op  quantity unit
0         1.0  >       0.0  USD
1         1.0  +   -2800.0  USD
2         1.0  +       1.0  SPX


In [ ]:
model = BS2Model()
price, stats = model.price(timetable, dataset)
print(f"price: {price:11.6f}")

price:  296.131604


A Contract that pays the best of SPX or RUT.

In [ ]:
events = [
    {
        "track": "",
        "time": 1.0,
        "op": ">",
        "quantity": 0,
        "unit": "USD",
    },
    {
        "track": "",
        "time": 1.0,
        "op": "+",
        "quantity": -2800.0,
        "unit": "USD",
    },
    {
        "track": "",
        "time": 1.0,
        "op": ">",
        "quantity": 1,
        "unit": "SPX",
    },
    {
        "track": "",
        "time": 1.0,
        "op": "+",
        "quantity": 2,  # as RUT is half the value
        "unit": "RUT",
    },
]
rainbow_timetable = timetable_from_dicts(events)
print(rainbow_timetable["events"].to_pandas())

  track  time op  quantity unit
0         1.0  >       0.0  USD
1         1.0  +   -2800.0  USD
2         1.0  >       1.0  SPX
3         1.0  +       2.0  RUT


In [ ]:
for corr in np.linspace(-1.0, 1.0, 5):
    dataset["BS2"]["CORR"] = corr
    price, stats = model.price(rainbow_timetable, dataset)
    print(f"corr: {corr:6.1f}     price: {price:11.6f}")

corr:   -1.0     price:  580.599907
corr:   -0.5     price:  535.841455
corr:    0.0     price:  489.638849
corr:    0.5     price:  431.710739
corr:    1.0     price:  296.131604


As expected, the 100% correlation matches the SPX only vanilla option. Otherwise, the option is worth higher as the correlation gets lower.